In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import re

module_path=os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import fasttext
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
import warnings
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import neattext.functions as nfx
 
warnings.filterwarnings(action='ignore')

from code_utils.utils import preprocess,aplatir

In [3]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', lines= True).dropna(subset=['year'])

In [4]:
wg_doi=pd.read_json('ipcc_wg.json')

In [5]:
df_ipcc=df_ipcc.merge(wg_doi,on='doi',how='left')

In [6]:
df_ipcc.columns

Index(['index', 'doi', 'title', 'year', 'countries', 'concepts', 'sdg',
       'topics', 'is_OA_available', 'title_OA', 'year_OA', 'authors_name',
       'rors', 'institutions_names', 'locations_names', 'locations_ids',
       'test_glutton', 'wg'],
      dtype='object')

In [7]:
dict_wg={'wg1':'science','wg2':'adaptation','wg2_cross':'adaptation','wg3':'mitigation'}

In [8]:
df_ipcc['wg_name']=df_ipcc['wg'].apply(lambda y: [dict_wg[str(x)] for x in y] if isinstance(y, list) else None)

In [9]:
df_ipcc=df_ipcc[['year','doi', 'title','topics','locations_names','locations_ids','wg_name']]

In [10]:
df_ipcc=df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']).drop_duplicates(subset=['doi'])

In [11]:
data_counts=pd.Series([int(x) for x in list(df_ipcc.year.dropna())]).value_counts()
data_counts=pd.Series(dict(data_counts)).sort_index()

building the learning dataset

In [12]:
df_all=df_ipcc

In [13]:
df_all.loc[:,'doi']=df_all.loc[:,'doi'].apply(lambda x: str(nfx.remove_stopwords(x)).replace('https://doi.org/','').lower())

In [14]:
df_all.loc[:,'title']=df_all.loc[:,'title'].apply(lambda x: str(nfx.remove_stopwords(x)).replace('}','').replace('{','').lower())

In [15]:
df_all.loc[:,'topics']=df_all.loc[:,'topics'].apply(lambda x: nfx.remove_stopwords(' '.join(x)).lower())

In [16]:
df_all

,year,doi,title,topics,locations_names,locations_ids,wg_name
0,2019.0,10.3389/978-2-88963-118-6,oceanobs'19: ocean opportunity. volume 1,marine biodiversity ecosystem functioning,Frontiers research topics,1664-8714,[science]
2,1900.0,10.1002/andp.19003081208,"\""uber die bedeutung des wasserdampfes und der...",stratospheric chemistry climate change impacts...,Annalen der Physik,0003-3804,[science]
3,1964.0,10.3402/tellusa.v16i1.8885,the parameters atmospheric turbidity,dynamics ocean surface waves wind interaction,Tellus A,0280-6495,[science]
4,1929.0,10.1080/20014422.1929.11880498,on atmospheric transmission sun radiation dust...,aerosols' impact climate hydrological cycle,Geografiska Annaler,1651-3215,[science]
5,2013.0,10.1002/rog.20022,a review global ocean temperature observations...,oceanic modeling circulation studies global se...,Reviews of Geophysics,1944-9208,"[science, adaptation]"
...,...,...,...,...,...,...,...
52877,2017.0,10.1007/s11625-017-0498-1,facilitating data-intensive approaches innovat...,smart cities: innovations challenges living la...,Sustainability Science,1862-4057,[mitigation]
52878,2021.0,10.1016/j.worlddev.2020.105249,china's green transformation eco-industrial parks,industrial symbiosis eco-industrial parks conc...,World Development,0305-750X,[mitigation]
52880,2018.0,10.1016/j.jclepro.2018.05.194,"water-energy-food nexus: concepts, questions m...","integrated management water, energy, food reso...",Journal of Cleaner Production,0959-6526,[mitigation]
52881,2018.0,10.1016/j.apenergy.2017.07.036,water-food-energy nexus optimization approach ...,"integrated management water, energy, food reso...",Applied Energy,0306-2619,[mitigation]


In [17]:
df_all.columns

Index(['year', 'doi', 'title', 'topics', 'locations_names', 'locations_ids',
       'wg_name'],
      dtype='object')

using fasttext

In [18]:
df_all['label']=df_all['wg_name'].apply(lambda x: ' '.join(['__label__' + str(y) for y in x])) 
df_all['category_description']=df_all['label']+' '+df_all['title']+' '+df_all['topics']+' '+df_all['locations_names']+' '+df_all['locations_ids']

In [19]:
train, test=train_test_split(df_all, test_size=0.2, random_state=42)

In [20]:
train.shape, test.shape

((38575, 9), (9644, 9))

In [21]:
train.to_csv("teds_wg.train", columns=["category_description"], index=False, header=False)
test.to_csv("teds_wg.test", columns=["category_description"], index=False, header=False)

In [22]:
model=fasttext.train_supervised(input="teds_wg.train", loss='ova')
model.test("teds_wg.test")

(7903, 0.8768821966341895, 0.7833163784333672)

In [23]:
model.save_model("fasttext_model_teds_wg.bin")

In [24]:
model=fasttext.load_model("fasttext_model_teds_wg.bin")

In [25]:
test_data=pd.read_csv("teds_wg.test", header=None, names=["text"])

In [ ]:
# pour multi labels
true_labels=[]
predicted_labels=[]

label_order=['__label__science', '__label__adaptation', '__label__mitigation']

for line in test_data["text"]:
    label, text=line.split(' ', 1)
    if '__label__not_ipcc' in label:
        labels_list=[0, 0, 0]  
    else:
        labels_list=[1 if x in label.split(' ') else 0 for x in label_order]
    true_labels.append(labels_list)
    
    text=text.strip()
    labels, probabilities=model.predict(text, k=-1, threshold=0.5)
    if '__label__not_ipcc' in labels:
        predicted_label=[0, 0, 0]  
    else:
        predicted_label=[1 if x in labels else 0 for x in label_order]
    predicted_labels.append(predicted_label)

In [26]:
# pour multi labels
true_labels = []
predicted_labels = []

for line in test_data["text"]:
    label, text = line.split(' ', 1)
    labels_list = [x.replace('__label__', '').strip() for x in label.split(',')]
    true_labels.append(labels_list)
    
    text = text.strip()
    labels, probabilities = model.predict(text, k=-1, threshold=0.5)
    predicted_labels.append([x.replace('__label__', '').strip() for x in labels])

mlb = MultiLabelBinarizer()
true_labels = mlb.fit_transform(true_labels)
predicted_labels = mlb.transform(predicted_labels)

In [27]:
predicted_labels

array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [28]:
true_labels

array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [31]:
#conf_matrix=confusion_matrix(true_labels, predicted_labels)

In [30]:
labels=mlb.classes_

for i, label in enumerate(labels):
    conf_matrix=confusion_matrix(true_labels[:, i], predicted_labels[:, i])
    print(f"\nmatrice de confusion pour '{label}':")
    print(conf_matrix)
    accuracy=accuracy_score(true_labels[:, i], predicted_labels[:, i])
    recall=recall_score(true_labels[:, i], predicted_labels[:, i])
    f1=f1_score(true_labels[:, i], predicted_labels[:, i])
    print(f"Accuracy pour '{label}': {accuracy:.4f}")
    print(f"Recall pour '{label}': {recall:.4f}")
    print(f"F1 score pour '{label}': {f1:.4f}")


matrice de confusion pour 'adaptation':
[[3560  914]
 [ 593 4577]]
Accuracy pour 'adaptation': 0.8437
Recall pour 'adaptation': 0.8853
F1 score pour 'adaptation': 0.8586

matrice de confusion pour 'mitigation':
[[7098  575]
 [ 301 1670]]
Accuracy pour 'mitigation': 0.9092
Recall pour 'mitigation': 0.8473
F1 score pour 'mitigation': 0.7922

matrice de confusion pour 'science':
[[6818  323]
 [ 446 2057]]
Accuracy pour 'science': 0.9203
Recall pour 'science': 0.8218
F1 score pour 'science': 0.8425


In [32]:
accuracy_score(true_labels, predicted_labels)

0.7983201990875155

In [33]:
recall_score(true_labels, predicted_labels, average="weighted")

0.861053504769805

In [34]:
f1_score(true_labels, predicted_labels, average="weighted")

0.8408822273933394

In [35]:
t=test.iloc[0,:]

In [36]:
t

year                                                               2020.0
doi                                                  10.1029/2020gl087014
title                   interannual variability outflow weddell sea water
topics                  oceanic modeling circulation studies climate c...
locations_names                              Geophysical Research Letters
locations_ids                                                   0094-8276
wg_name                                                         [science]
label                                                    __label__science
category_description    __label__science interannual variability outfl...
Name: 8360, dtype: object

In [37]:
model.predict(nfx.remove_stopwords(f"{t.title} {t.topics} {t.locations_names} {t.locations_ids}"), k=-1)

(('__label__science', '__label__adaptation', '__label__mitigation'),
 array([8.63401651e-01, 2.01823220e-01, 1.00000034e-05]))

In [38]:
model.predict(nfx.remove_stopwords(f"{t.title} {t.topics} {t.locations_names} {t.locations_ids}"))

(('__label__science',), array([0.86340165]))